In [ ]:
from dotenv import load_dotenv


load_dotenv()

# Imports

In [ ]:
import json
import os

from libdc3.config import dc3_config
from libdc3.methods.acc_lumi_analyzer import AccLuminosityAnalyzer
from libdc3.methods.bril_actions import BrilActions
from libdc3.methods.json_producer import JsonProducer
from libdc3.methods.rr_actions import RunRegistryActions

# User input

In [ ]:
output_path = "./results/calls_1_23"

# Call specific numbers
cycles_list = [
    "Collisions2024_certification_call1",
    "Collisions2024_certification_call2",
    "Collisions2024_certification_call3",
    "Collisions2024_certification_call4",
    "Collisions2024_certification_call5",
    "Collisions2024_certification_call6",
    "Collisions2024_certification_call7",
    "Collisions2024_certification_call8",
    "Collisions2024_certification_call9",
    "Collisions2024_certification_call10",
    "Collisions2024_certification_call11",
    "collisions2024_certification_call12",
    "collisions2024_certification_call13",
    "collisions2024_certification_call14",
    "collisions2024_certification_call15",
    "Collisions2024_certification_call16",
    "Collisions2024_certification_call17",
    "Collisions2024_certification_call18",
    "Collisions2024_certification_call19",
    "Collisions2024_certification_call20",
    "Collisions2024_certification_call21",
    "Collisions2024_certification_call22",
    "Collisions2024_certification_call23",
]

# Run registry specific variables
rr_class_name = "Collisions24"
rr_dataset_name = "/PromptReco/Collisions2024/DQM"

# Bril specific variables
bril_brilws_version = "3.7.4"
bril_unit = "/ub"
bril_low_lumi_thr = 80000.0
bril_beamstatus = "STABLE BEAMS"
bril_amodetag = "PROTPHYS"
bril_normtag = "/cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_BRIL.json"

# Json production specific
ignore_hlt_emergency = False
prejson_oms_flags = ["beam1_present", "beam2_present", "beam1_stable", "beam2_stable"]
goldenjson_oms_flags = [
    "beam1_present",
    "beam2_present",
    "beam1_stable",
    "beam2_stable",
    "cms_active",
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
    "hbhea_ready",
    "hbheb_ready",
    "hbhec_ready",
    "hf_ready",
    "ho_ready",
]
goldenjson_rr_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "ecal-ecal",
    "ecal-es",
    "hcal-hcal",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "l1t-l1tcalo",
    "hlt-hlt",
    "egamma-egamma",
    "muon-muon",
    "jetmet-jetmet",
]
muonjson_oms_flags = [
    "beam1_present",
    "beam2_present",
    "beam1_stable",
    "beam2_stable",
    "cms_active",
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
]
muonjson_rr_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "hlt-hlt",
    "muon-muon",
]

# Lumiloss specific
target_lumiloss_unit = "/pb"
lumiloss_dcs_flags = [
    "bpix_ready",
    "fpix_ready",
    "tibtid_ready",
    "tecm_ready",
    "tecp_ready",
    "tob_ready",
    "hbhea_ready",
    "hbheb_ready",
    "hbhec_ready",
    "hf_ready",
    "ho_ready",
]
lumiloss_subsystems_flags = [
    "tracker-pixel",
    "tracker-strip",
    "tracker-track",
    "ecal-ecal",
    "ecal-es",
    "hcal-hcal",
    "csc-csc",
    "dt-dt",
    "l1t-l1tmu",
    "l1t-l1tcalo",
    "hlt-hlt",
    "egamma-egamma",
    "muon-muon",
    "jetmet-jetmet",
]
lumiloss_subdetectors_flags = {
    "PixelPhase1": ["tracker-pixel", "bpix_ready", "fpix_ready"],
    "SiStrip": ["tracker-strip", "tibtid_ready", "tecm_ready", "tecp_ready", "tob_ready"],
    "ECAL": ["ecal-ecal"],
    "ES": ["ecal-es"],
    "HCAL": ["hcal-hcal", "hbhea_ready", "hbheb_ready", "hbhec_ready", "hf_ready", "ho_ready"],
    "CSC": ["csc-csc"],
    "DT": ["dt-dt"],
    "L1T": ["l1t-l1tcalo", "l1t-l1tmu"],
    "HLT": ["hlt-hlt"],
    "Tracking": ["tracker-track"],
    "MuonPOG": ["muon-muon"],
    "JetMET": ["jetmet-jetmet"],
    "EGamma": ["egamma-egamma"],
}

# Acc. luminosity variables
target_acclumi_unit = "/fb"
acc_lumi_year = 2024
acc_lumi_energy_label = "13.6 TeV"
acc_lumi_additional_label_on_plot = "CMS Preliminary Offline Luminosity"  # None if no label

# Configure dc3 sensitive variables

In [ ]:
dc3_config.set_keytab_usr(os.getenv("KEYTAB_USR"))
dc3_config.set_keytab_pwd(os.getenv("KEYTAB_PWD"))
dc3_config.set_auth_cert_path(os.getenv("AUTH_CERT"))
dc3_config.set_auth_key_path(os.getenv("AUTH_CERT_KEY"))

# Setup output path if not exists

In [ ]:
os.makedirs(output_path, exist_ok=True)

# Fetch RR and OMS lumisections for all runs

In [ ]:
rra = RunRegistryActions(class_name=rr_class_name, dataset_name=rr_dataset_name)
runs_in_all_cycles = rra.fetch_runs_in_cycles(cycles_list=cycles_list)
offline_lumis = rra.multi_fetch_rr_oms_joint_lumis(run_list=runs_in_all_cycles)

# Instantiate the JSON producer

In [ ]:
producer = JsonProducer(rr_oms_lumis=offline_lumis, ignore_hlt_emergency=ignore_hlt_emergency)

# Generate the goldenJSON

In [ ]:
gjson = producer.generate(goldenjson_oms_flags, goldenjson_rr_flags)

# Save
base_path = os.path.join(output_path, "jsons")
os.makedirs(base_path, exist_ok=True)
fpath = os.path.join(base_path, "golden.json")
with open(fpath, "w") as f:
    json.dump(gjson, f, ensure_ascii=False, indent=4)

# Display
gjson

# Generate the muonJSON

In [ ]:
mjson = producer.generate(muonjson_oms_flags, muonjson_rr_flags)

# Save
base_path = os.path.join(output_path, "jsons")
os.makedirs(base_path, exist_ok=True)
fpath = os.path.join(base_path, "muon.json")
with open(fpath, "w") as f:
    json.dump(mjson, f, ensure_ascii=False, indent=4)

# Display
mjson

# Fetch BRIL lumisections for run list

In [ ]:
min_run = min(runs_in_all_cycles)
max_run = max(runs_in_all_cycles)
ba = BrilActions(
    brilws_version=bril_brilws_version,
    unit=bril_unit,
    low_lumi_thr=bril_low_lumi_thr,
    beamstatus=bril_beamstatus,
    amodetag=bril_amodetag,
    normtag=bril_normtag,
)
bril_lumis = ba.fetch_lumis(begin=min_run, end=max_run).get("detailed")

# Generate acc luminosity plots for goldenJSON

In [ ]:
acc_lumi_plots_path = os.path.join(output_path, "acc_lumi/golden")
os.makedirs(acc_lumi_plots_path, exist_ok=True)

acc_lumi = AccLuminosityAnalyzer(
    dc_json=gjson,
    bril_lumis=bril_lumis,
    bril_amodetag=bril_amodetag,
    bril_unit=bril_unit,
    target_unit=target_acclumi_unit,
    year=acc_lumi_year,
    plot_energy_label=acc_lumi_energy_label,
    output_path=acc_lumi_plots_path,
    additional_label_on_plot=acc_lumi_additional_label_on_plot,
)
acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
del acc_lumi

# In case we want to test only the AccLuminosityAnalyzer alone
# import pickle

# with open("AccLuminosityAnalyzer-inputs.pickle", "wb") as f:
#     inputs = {
#         "dc_json": gjson,
#         "bril_lumis": bril_lumis,
#         "bril_amodetag": bril_amodetag,
#         "bril_unit": bril_unit,
#         "target_unit": target_acclumi_unit,
#         "year": acc_lumi_year,
#         "beam_energy": acc_lumi_beam_energy,
#         "output_path": acc_lumi_plots_path,
#         "additional_label_on_plot": acc_lumi_additional_label_on_plot
#     }
#     pickle.dump(inputs, f, protocol=pickle.HIGHEST_PROTOCOL)

# Generate acc luminosity plots for goldenJSON

In [ ]:
acc_lumi_plots_path = os.path.join(output_path, "acc_lumi/muon")
os.makedirs(acc_lumi_plots_path, exist_ok=True)

acc_lumi = AccLuminosityAnalyzer(
    dc_json=mjson,
    bril_lumis=bril_lumis,
    bril_amodetag=bril_amodetag,
    bril_unit=bril_unit,
    target_unit=target_acclumi_unit,
    year=acc_lumi_year,
    plot_energy_label=acc_lumi_energy_label,
    output_path=acc_lumi_plots_path,
    additional_label_on_plot=acc_lumi_additional_label_on_plot,
)
acc_lumi.plot_acc_lumi_by_day(save_pdf=True)
acc_lumi.plot_acc_lumi_by_week(save_pdf=True)
del acc_lumi, bril_lumis